In [2]:
# import pandas as pd
# import numpy as np
# import lightgbm as lgb
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_squared_error

# # Load data
# train = pd.read_csv('/kaggle/input/prediction-interval-competition-ii-house-price/train.csv')
# test = pd.read_csv('/kaggle/input/prediction-interval-competition-ii-house-price/test.csv')
# sample_submission = pd.read_csv('/kaggle/input/prediction-interval-competition-ii-house-price/sample_submission.csv')

# # Save test IDs
# test_ids = test['id']

# # Prepare data
# y = np.log1p(train['SalePrice'])  # log-transform the target
# X = train.drop(columns=['SalePrice', 'id'])
# X_test = test.drop(columns=['id'])

# # LightGBM quantile regression function
# def train_lgb(X_train, y_train, alpha):
#     params = {
#         'objective': 'quantile',
#         'metric': 'quantile',
#         'alpha': alpha,
#         'learning_rate': 0.03,
#         'num_leaves': 64,
#         'min_data_in_leaf': 10,
#         'feature_fraction': 0.8,
#         'bagging_fraction': 0.8,
#         'bagging_freq': 1,
#         'verbose': -1,
#         'seed': 42
#     }
#     train_data = lgb.Dataset(X_train, label=y_train)
#     model = lgb.train(params, train_data, num_boost_round=1000)
#     return model

# # Cross-validation
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# preds_lower = np.zeros(X_test.shape[0])
# preds_upper = np.zeros(X_test.shape[0])

# winkler_scores = []

# for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
#     print(f"🔁 Fold {fold+1}")

#     X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
#     y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

#     model_l = train_lgb(X_tr, y_tr, alpha=0.05)
#     model_u = train_lgb(X_tr, y_tr, alpha=0.95)

#     val_l = model_l.predict(X_val)
#     val_u = model_u.predict(X_val)
#     y_val_true = np.expm1(y_val)  # back-transform
#     val_l_exp = np.expm1(val_l)
#     val_u_exp = np.expm1(val_u)

#     # Compute local winkler score
#     def winkler_score(y_true, y_lower, y_upper, alpha=0.1):
#         score = []
#         for yt, l, u in zip(y_true, y_lower, y_upper):
#             if yt < l:
#                 s = (u - l) + (2 / alpha) * (l - yt)
#             elif yt > u:
#                 s = (u - l) + (2 / alpha) * (yt - u)
#             else:
#                 s = u - l
#             score.append(s)
#         return np.mean(score)

#     score = winkler_score(y_val_true, val_l_exp, val_u_exp)
#     winkler_scores.append(score)
#     print(f"📏 Winkler Score (Fold {fold+1}): {score:.2f}")

#     # Test predictions
#     preds_lower += np.expm1(model_l.predict(X_test)) / kf.n_splits
#     preds_upper += np.expm1(model_u.predict(X_test)) / kf.n_splits

# print(f"\n✅ Avg Winkler Score: {np.mean(winkler_scores):.2f}")

# # Prepare submission
# submission = pd.DataFrame({
#     'id': test_ids,
#     'pi_lower': preds_lower,
#     'pi_upper': preds_upper
# })

# submission.to_csv("submission.csv", index=False)
# print("✅ submission.csv created!")


In [3]:
# !pip install lgb


In [4]:
# import pandas as pd
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn import RandomForestQuantileRegressor
# from sklearn.model_selection import train_test_split, KFold
# from sklearn.preprocessing import StandardScaler

# # Load the data
# train = pd.read_csv("/kaggle/input/prediction-interval-competition-ii-house-price/train.csv")
# test = pd.read_csv("/kaggle/input/prediction-interval-competition-ii-house-price/test.csv")
# sample_submission = pd.read_csv("/kaggle/input/prediction-interval-competition-ii-house-price/sample_submission.csv")

# # Prepare features and log-transform target
# X = train.drop(columns=['id', 'SalePrice'])
# y = np.log1p(train['SalePrice'])  # log-transform target for stability
# X_test = test.drop(columns=['id'])
# test_ids = test['id']

# # Train-test split for local validation
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train Quantile Regression Forest
# model = RandomForestQuantileRegressor(n_estimators=100, min_samples_leaf=2, random_state=42, n_jobs=-1)
# model.fit(X_train, y_train)

# # Predict lower and upper bounds on validation
# val_lower = np.expm1(model.predict(X_val, quantile=5))
# val_upper = np.expm1(model.predict(X_val, quantile=95))
# y_val_true = np.expm1(y_val)

# # Winkler score function
# def winkler_score(y_true, y_lower, y_upper, alpha=0.1):
#     score = []
#     for yt, l, u in zip(y_true, y_lower, y_upper):
#         if yt < l:
#             s = (u - l) + (2 / alpha) * (l - yt)
#         elif yt > u:
#             s = (u - l) + (2 / alpha) * (yt - u)
#         else:
#             s = u - l
#         score.append(s)
#     return np.mean(score)

# # Evaluate Winkler score
# winkler = winkler_score(y_val_true, val_lower, val_upper)
# print(f"📏 Local Winkler Score (Validation): {winkler:.2f}")

# # Predict on test data
# test_lower = np.expm1(model.predict(X_test, quantile=5))
# test_upper = np.expm1(model.predict(X_test, quantile=95))

# # Create submission file
# submission = pd.DataFrame({
#     'id': test_ids,
#     'pi_lower': test_lower,
#     'pi_upper': test_upper
# })
# submission.to_csv("submission.csv", index=False)
# print("✅ Submission saved as 'submission.csv'")


In [5]:
# import pandas as pd
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error

# # Load data
# train = pd.read_csv("dataset.csv")
# test = pd.read_csv("test.csv")
# submission_format = pd.read_csv("sample_submission.csv")

# # Features and target
# X = train.drop(columns=["id", "SalePrice"])
# y = np.log1p(train["SalePrice"])  # log-transform target
# X_test = test.drop(columns=["id"])
# test_ids = test["id"]

# # Train-validation split
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Base RandomForest settings
# rf_params = {
#     "n_estimators": 100,
#     "min_samples_leaf": 2,
#     "random_state": 42,
#     "n_jobs": -1
# }

# # Train two separate models for lower (5%) and upper (95%) bounds
# rf_lower = RandomForestRegressor(**rf_params)
# rf_upper = RandomForestRegressor(**rf_params)

# rf_lower.fit(X_train, y_train)
# rf_upper.fit(X_train, y_train)

# # Get predictions from all trees for each sample (for quantiles)
# def predict_quantiles(model, X, q):
#     all_preds = np.stack([tree.predict(X) for tree in model.estimators_], axis=1)
#     return np.percentile(all_preds, q * 100, axis=1)

# # Predict on validation set
# val_lower = np.expm1(predict_quantiles(rf_lower, X_val, q=0.05))
# val_upper = np.expm1(predict_quantiles(rf_upper, X_val, q=0.95))
# y_val_true = np.expm1(y_val)

# # Winkler Score
# def winkler_score(y_true, y_lower, y_upper, alpha=0.1):
#     score = []
#     for yt, l, u in zip(y_true, y_lower, y_upper):
#         if yt < l:
#             s = (u - l) + (2 / alpha) * (l - yt)
#         elif yt > u:
#             s = (u - l) + (2 / alpha) * (yt - u)
#         else:
#             s = u - l
#         score.append(s)
#     return np.mean(score)

# # Evaluate
# winkler = winkler_score(y_val_true, val_lower, val_upper)
# print(f"📏 Local Winkler Score (Validation): {winkler:.2f}")

# # Predict on test set
# test_lower = np.expm1(predict_quantiles(rf_lower, X_test, q=0.05))
# test_upper = np.expm1(predict_quantiles(rf_upper, X_test, q=0.95))

# # Save submission
# submission = pd.DataFrame({
#     "id": test_ids,
#     "pi_lower": test_lower,
#     "pi_upper": test_upper
# })
# submission.to_csv("submission.csv", index=False)
# print("✅ submission.csv saved!")


In [6]:
# import pandas as pd
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split

# # Load files
# train = pd.read_csv("dataset.csv")
# test = pd.read_csv("test.csv")
# sample_submission = pd.read_csv("sample_submission.csv")

# # Extract features and target
# X = train.drop(columns=["id", "sale_price"])
# y = np.log1p(train["sale_price"])  # log-transform to stabilize variance
# X_test = test.drop(columns=["id"])
# test_ids = test["id"]

# # Train-validation split for local evaluation
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define model parameters
# rf_params = {
#     "n_estimators": 100,
#     "min_samples_leaf": 2,
#     "random_state": 42,
#     "n_jobs": -1
# }

# # Initialize models
# model_lower = RandomForestRegressor(**rf_params)
# model_upper = RandomForestRegressor(**rf_params)

# # Fit both models
# model_lower.fit(X_train, y_train)
# model_upper.fit(X_train, y_train)

# # Function to extract quantile predictions
# def predict_quantiles(model, X, q):
#     all_preds = np.stack([tree.predict(X) for tree in model.estimators_], axis=1)
#     return np.percentile(all_preds, q * 100, axis=1)

# # Predict on validation set
# val_lower = np.expm1(predict_quantiles(model_lower, X_val, 0.05))  # 5th percentile
# val_upper = np.expm1(predict_quantiles(model_upper, X_val, 0.95))  # 95th percentile
# y_val_true = np.expm1(y_val)

# # Winkler Score implementation
# def winkler_score(y_true, y_lower, y_upper, alpha=0.1):
#     score = []
#     for yt, l, u in zip(y_true, y_lower, y_upper):
#         if yt < l:
#             s = (u - l) + (2 / alpha) * (l - yt)
#         elif yt > u:
#             s = (u - l) + (2 / alpha) * (yt - u)
#         else:
#             s = u - l
#         score.append(s)
#     return np.mean(score)

# # Evaluate Winkler Score on validation
# winkler = winkler_score(y_val_true, val_lower, val_upper)
# print(f"📏 Winkler Score (Validation): {winkler:.2f}")

# # Predict intervals on test set
# test_lower = np.expm1(predict_quantiles(model_lower, X_test, 0.05))
# test_upper = np.expm1(predict_quantiles(model_upper, X_test, 0.95))

# # Format submission
# submission = pd.DataFrame({
#     "id": test_ids,
#     "pi_lower": test_lower,
#     "pi_upper": test_upper
# })

# # Save file
# submission.to_csv("submission.csv", index=False)
# print("✅ submission.csv generated!")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Load data
train = pd.read_csv("dataset.csv")
test = pd.read_csv("test.csv")

# Extract target and drop from training features
X = train.drop(columns=["id", "sale_price"])
y = np.log1p(train["sale_price"])  # log-transform target
X_test = test.drop(columns=["id"])
test_ids = test["id"]

# --- Handle date columns ---
def process_dates(df):
    for col in df.columns:
        if df[col].dtype == "object":
            try:
                df[col] = pd.to_datetime(df[col])
            except:
                continue
    for col in df.select_dtypes(include="datetime").columns:
        df[f"{col}_year"] = df[col].dt.year
        df[f"{col}_month"] = df[col].dt.month
        df[f"{col}_day"] = df[col].dt.day
        df.drop(columns=[col], inplace=True)
    return df

X = process_dates(X)
X_test = process_dates(X_test)

# --- Encode categorical variables ---
X = pd.get_dummies(X, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Align test set columns with training set
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# --- Train-validation split ---
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Random Forest model settings ---
rf_params = {
    "n_estimators": 100,
    "min_samples_leaf": 2,
    "random_state": 42,
    "n_jobs": -1
}

# --- Define models ---
model_lower = RandomForestRegressor(**rf_params)
model_upper = RandomForestRegressor(**rf_params)

# --- Train models ---
model_lower.fit(X_train, y_train)
model_upper.fit(X_train, y_train)

# --- Function to predict quantiles manually ---
def predict_quantiles(model, X, q):
    all_preds = np.stack([tree.predict(X) for tree in model.estimators_], axis=1)
    return np.percentile(all_preds, q * 100, axis=1)

# --- Predict on validation set ---
val_lower = np.expm1(predict_quantiles(model_lower, X_val, 0.05))
val_upper = np.expm1(predict_quantiles(model_upper, X_val, 0.95))
y_val_true = np.expm1(y_val)

# --- Winkler Score ---
def winkler_score(y_true, y_lower, y_upper, alpha=0.1):
    score = []
    for yt, l, u in zip(y_true, y_lower, y_upper):
        if yt < l:
            s = (u - l) + (2 / alpha) * (l - yt)
        elif yt > u:
            s = (u - l) + (2 / alpha) * (yt - u)
        else:
            s = u - l
        score.append(s)
    return np.mean(score)

# Evaluate
winkler_val = winkler_score(y_val_true, val_lower, val_upper)
print(f"📏 Winkler Score (Validation): {winkler_val:.2f}")

# --- Predict on test set ---
test_lower = np.expm1(predict_quantiles(model_lower, X_test, 0.05))
test_upper = np.expm1(predict_quantiles(model_upper, X_test, 0.95))

# --- Create submission file ---
submission = pd.DataFrame({
    "id": test_ids,
    "pi_lower": test_lower,
    "pi_upper": test_upper
})

submission.to_csv("submission.csv", index=False)
print("✅ Submission file 'submission.csv' created!")
